In [25]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import csv

In [2]:
# Credenciales de Twitter
USERNAME = 'ScrapeoUPM'
PASSWORD = 'scrapeafutbol'
EMAIL = 'oscar.marin@alumnos.upm.es'

SCROLL_PAUSE_TIME = 3  # Tiempo de espera para que carguen nuevos tweets

In [48]:
def login(driver, username, email, password):
    
    # Navegar a la página de inicio de sesión de Twitter
    driver.get('https://twitter.com/login')
    time.sleep(7)  # Esperar a que cargue la página

    # Buscar el campo del nombre de usuario e ingresar el nombre de usuario o correo
    username_input = driver.find_element(By.NAME, 'text')
    username_input.send_keys(username)
    username_input.send_keys(Keys.RETURN) #Presionar tecla enter
    time.sleep(2)
    
    try:
        # Buscar el campo de confirmación de correo
        email_input = driver.find_element(By.NAME, 'text')
        email_input.send_keys(email)
        email_input.send_keys(Keys.RETURN)
        time.sleep(2)
    except:
        pass
    
    # Buscar el campo de contraseña e ingresar la contraseña
    password_input = driver.find_element(By.NAME, 'password')
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)

def scrap_n_tweets(driver, n_tweets: int, word_search: str, date_range: tuple):

    word_search = word_search.replace(' ', '%20')
    date_range = f'%20until%3A{date_range[1]}%20since%3A{date_range[0]}'

    # Navegar a la página de búsqueda de Twitter
    driver.get(f'https://twitter.com/search?q={word_search}{date_range}')
    time.sleep(5)  # Esperar a que cargue la página

    n = 0
    tweets = []
    while n < n_tweets:
        # Extraer los tweets visibles en este momento
        new_tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        # Imprimir el texto de los tweets extraídos
        for tweet in new_tweets:
            tweet_text = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
            tweet_date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
            tweet_lang  = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]').get_attribute('lang')
            tweets.append([word_search, tweet_text.text, tweet_date, tweet_lang])
            n += 1
            if n == n_tweets:
                break

        # Hacer scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

    return tweets

def scrap_all_tweets(driver, word_search: str, date_range: tuple):
    
    word_search = word_search.replace(' ', '%20')
    date_range = f'%20until%3A{date_range[1]}%20since%3A{date_range[0]}'

    # Navegar a la página de búsqueda de Twitter
    driver.get(f'https://twitter.com/search?q={word_search}{date_range}')
    time.sleep(5)  # Esperar a que cargue la página
  
    last_height = driver.execute_script("return document.body.scrollHeight")
    tweets = []
    while True:
        # Extraer los tweets visibles en este momento
        new_tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        # Imprimir el texto de los tweets extraídos
        for tweet in new_tweets:
            tweet_text = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
            tweet_date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
            tweet_lang  = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]').get_attribute('lang')
            tweets.append([word_search, tweet_text.text, tweet_date, tweet_lang])
            
        # Hacer scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return tweets

In [30]:
def get_weeks(since, until):
    weeks = []
    actual_date = datetime.strptime(since, '%Y-%m-%d')
    final_date = datetime.strptime(until, '%Y-%m-%d')

    while actual_date + timedelta(days=7) < final_date:
        # Definir el rango de cada semana
        week_end = actual_date + timedelta(days=6) # Día actual + 6 días = 7 días

        weeks.append((actual_date.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d')))
        # Avanzar a la siguiente semana
        actual_date = week_end + timedelta(days=1) # Empieza el siguiente día para no repetir

    weeks.append((actual_date.strftime('%Y-%m-%d'), until))
    return weeks

get_weeks('2024-09-30', '2024-11-06')

[('2024-09-30', '2024-10-06'),
 ('2024-10-07', '2024-10-13'),
 ('2024-10-14', '2024-10-20'),
 ('2024-10-21', '2024-10-27'),
 ('2024-10-28', '2024-11-03'),
 ('2024-11-04', '2024-11-06')]

In [50]:
player = 'Ferran Torres'
since = '2023-08-01'
until = '2024-07-15'
n_tweets = 100

chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecuta Chrome en modo headless

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
login(driver, USERNAME, EMAIL, PASSWORD)

weeks = get_weeks(since, until)

all_tweets = []
for since, until in weeks:
    print(f'Scraping tweets from {since} to {until}')
    tweets = scrap_all_tweets(driver, player, (since, until))
    print(f'Found {len(tweets)} tweets')
    all_tweets.extend(tweets)

# Crear un archivo CSV con los tweets extraídos
with open(f'/raw/{player.replace(' ', '_')}_tweets.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['text', 'date', 'lang'])
    for tweet in all_tweets:
        writer.writerow(tweet)

# Cerrar el navegador
driver.quit()
tweets

Scraping tweets from 2023-08-01 to 2023-08-07
Found 112 tweets
Scraping tweets from 2023-08-08 to 2023-08-14
Found 158 tweets
Scraping tweets from 2023-08-15 to 2023-08-21
Found 163 tweets
Scraping tweets from 2023-08-22 to 2023-08-28
Found 31 tweets
Scraping tweets from 2023-08-29 to 2023-09-04
Found 28 tweets
Scraping tweets from 2023-09-05 to 2023-09-11
Found 30 tweets
Scraping tweets from 2023-09-12 to 2023-09-18
Found 11 tweets
Scraping tweets from 2023-09-19 to 2023-09-25
Found 0 tweets
Scraping tweets from 2023-09-26 to 2023-10-02
Found 0 tweets
Scraping tweets from 2023-10-03 to 2023-10-09
Found 0 tweets
Scraping tweets from 2023-10-10 to 2023-10-16
Found 0 tweets
Scraping tweets from 2023-10-17 to 2023-10-23
Found 0 tweets
Scraping tweets from 2023-10-24 to 2023-10-30
Found 0 tweets
Scraping tweets from 2023-10-31 to 2023-11-06
Found 0 tweets
Scraping tweets from 2023-11-07 to 2023-11-13
Found 0 tweets
Scraping tweets from 2023-11-14 to 2023-11-20
Found 0 tweets
Scraping tweet

KeyboardInterrupt: 